<a href="https://colab.research.google.com/github/Sirenomanx/Repositorio-POO/blob/main/proyectofinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00


In [1]:
%%writefile app.py
# ------------------ LIBRERÍAS ------------------
import streamlit as st                        # Interfaz web
from datetime import datetime                # Fechas y tiempos
import pickle                                # Guardar y cargar datos
import random                                # Selección aleatoria de mensajes
import smtplib                               # Envío de correos
from email.mime.text import MIMEText         # Estructura del mensaje de correo

# ------------------ CLASES ------------------

# Clase que representa a un contacto
class Contacto:
    def __init__(self, nombre, fecha_nacimiento, email, msj_custom=None):
        self.nombre = nombre
        self.fecha_nacimiento = datetime.strptime(fecha_nacimiento, "%Y-%m-%d")  # Convierte string a objeto datetime
        self.email = email
        self.msj_custom = msj_custom  # Mensaje personalizado (opcional)

    # Calcula los días que faltan para su próximo cumpleaños
    def dias_para_cumple(self):
        hoy = datetime.now().date()  # Fecha actual
        cumple_este_anio = self.fecha_nacimiento.replace(year=hoy.year).date()
        if cumple_este_anio < hoy:  # Si ya pasó este año, se calcula para el siguiente
            cumple_este_anio = self.fecha_nacimiento.replace(year=hoy.year + 1).date()
        return (cumple_este_anio - hoy).days  # Diferencia en días

# Clase para manejar un mensaje de cumpleaños
class Mensaje:
    def __init__(self, contenido):
        self.contenido = contenido

    def mostrar(self):
        return self.contenido  # Devuelve el texto del mensaje

# Clase que gestiona toda la lógica de cumpleaños, mensajes y correos
class GestorCumples:
    def __init__(self):
        self.contactos = self.cargar_datos("contactos.pkl")  # Carga contactos guardados
        self.mensajes = self.cargar_datos("mensajes.pkl")    # Carga mensajes genéricos guardados

    # Guarda datos (contactos o mensajes) en un archivo .pkl
    def guardar_datos(self, archivo, datos):
        with open(archivo, "wb") as f:
            pickle.dump(datos, f)

    # Carga datos desde un archivo .pkl
    def cargar_datos(self, archivo):
        try:
            with open(archivo, "rb") as f:
                return pickle.load(f)
        except:
            return []  # Si falla, retorna lista vacía

    # Agrega un nuevo contacto a la lista
    def agregar_contacto(self, nombre, fecha_nacimiento, email, msj_custom):
        contacto = Contacto(nombre, fecha_nacimiento, email, msj_custom)
        self.contactos.append(contacto)
        self.guardar_datos("contactos.pkl", self.contactos)

    # Agrega un nuevo mensaje genérico
    def agregar_mensaje_generico(self, contenido):
        self.mensajes.append(Mensaje(contenido))
        self.guardar_datos("mensajes.pkl", self.mensajes)

    # Devuelve el mensaje adecuado para el contacto
    def obtener_mensaje_para(self, contacto):
        if contacto.msj_custom:
            return contacto.msj_custom  # Prioriza el personalizado
        elif self.mensajes:
            return random.choice(self.mensajes).mostrar()  # Elige uno aleatorio
        else:
            return "¡Feliz cumpleaños!"  # Mensaje por defecto

    # Envío de correo a través de SMTP usando una cuenta de Gmail
    def enviar_correo(self, destino, mensaje):
        try:
            remitente = "luis.vega3336@alumnos.udg.mx"   # Tu correo
            clave = "zfhp ncjv bcyz tgoo"               # Contraseña de aplicación generada
            msg = MIMEText(mensaje)
            msg['Subject'] = "🎉 ¡Feliz cumpleaños!"
            msg['From'] = remitente
            msg['To'] = destino

            # Conexión segura al servidor de Gmail y envío
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
                server.login(remitente, clave)
                server.send_message(msg)

            return True  # Si se envía correctamente
        except Exception as e:
            return f"Error al enviar: {e}"  # Devuelve el error en caso de fallo

# ------------------ INTERFAZ CON STREAMLIT ------------------

gestor = GestorCumples()  # Instancia del gestor principal

# Título principal de la app
st.title("Gestor de Cumpleaños")

# Menú lateral con opciones
menu = st.sidebar.radio("Opciones", ["Ver cumpleaños", "Agregar contacto", "Mensajes genéricos"])

# Opción: Agregar contacto
if menu == "Agregar contacto":
    st.header("Agregar nuevo contacto")
    with st.form("form_contacto"):
        nombre = st.text_input("Nombre")
        fecha_nacimiento = st.date_input("Fecha de nacimiento").strftime("%Y-%m-%d")
        email = st.text_input("Correo electrónico")
        msj_custom = st.text_area("Mensaje personalizado (opcional)")
        enviado = st.form_submit_button("Guardar")
        if enviado:
            gestor.agregar_contacto(nombre, fecha_nacimiento, email, msj_custom)
            st.success("Contacto guardado correctamente.")

# Opción: Mensajes genéricos
elif menu == "Mensajes genéricos":
    st.header("Mensajes genéricos")
    # Muestra los mensajes existentes
    for i, m in enumerate(gestor.mensajes):
        st.write(f"{i+1}. {m.mostrar()}")

    # Permite agregar uno nuevo
    nuevo = st.text_area("Agregar nuevo mensaje")
    if st.button("Agregar mensaje"):
        gestor.agregar_mensaje_generico(nuevo)
        st.success("Mensaje agregado.")

# Opción: Ver cumpleaños
elif menu == "Ver cumpleaños":
    st.header("Próximos cumpleaños")

    # Si no hay contactos, muestra aviso
    if not gestor.contactos:
        st.info("No hay contactos registrados.")

    # Muestra lista de contactos y permite enviarles correos
    for c in gestor.contactos:
        dias = c.dias_para_cumple()
        st.markdown(f"**{c.nombre}** ({c.email}) — en **{dias} día(s)**")
        mensaje = gestor.obtener_mensaje_para(c)
        st.markdown(f"> {mensaje}")

        # Botón para enviar correo
        if st.button(f"Enviar correo a {c.nombre}", key=c.email):
            resultado = gestor.enviar_correo(c.email, mensaje)
            if resultado is True:
                st.success("Correo enviado correctamente.")
            else:
                st.error(resultado)





Writing app.py


In [4]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 22 packages in 3s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [5]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.237.55.53
⠙your url is: https://smooth-kings-fly.loca.lt
